In [27]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as opt
import math

from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyarrow.csv as csv
import pyarrow.json as json

from enum import Enum
import abc
from typing import List, Dict, Tuple, Any, Union

In [28]:
sys.path.append('/groups/icecube/cyan/Utils')
from PlotUtils import setMplParam, getColour, getHistoParam 
# getHistoParam:
# Nbins, binwidth, bins, counts, bin_centers = 
from DB_lister import list_content, list_tables
from ExternalFunctions import nice_string_output, add_text_to_ax
setMplParam()

In [29]:
sys.path.append('/groups/icecube/cyan/factory/DOMification')
from Enum.Flavour import Flavour
from Enum.EnergyRange import EnergyRange

In [30]:
root_dir = "/lustre/hpc/project/icecube/HE_Nu_Aske_Oct2024/PMTfied_filtered_second_round/Snowstorm/CC_CRclean_IntraTravelDistance_250/"
root_dir_32 = root_dir + "22032/"
dir_32_1 = root_dir_32 + "1/"
truth_32_1 = root_dir_32 + "truth_1.parquet"
pmt_32_1_1 = dir_32_1 + "PMTfied_1.parquet"
pmt_32_1_2 = dir_32_1 + "PMTfied_2.parquet"

In [31]:
def convertParquetToDF(file:str) -> pd.DataFrame:
    table = pq.read_table(file)
    df = table.to_pandas()
    return df

In [38]:
def show_offset_edges(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies rows where the `offset` decreases or stays the same compared to the previous row.
    Useful for checking whether offset resets appropriately or has unexpected drops.
    """
    df = df.reset_index(drop=True)
    offset_diff = df['offset'].diff()
    mask = offset_diff < 0
    return df.loc[mask]

def show_shard_edges(df: pd.DataFrame) -> pd.DataFrame:
    """
    Identifies the rows where `shard_no` changes.
    Helpful to observe shard boundaries and investigate the offset reset logic.
    """
    df = df.reset_index(drop=True)
    shard_change = df['shard_no'].ne(df['shard_no'].shift())
    return df.loc[shard_change]


In [32]:
df_truth_32_1 = convertParquetToDF(truth_32_1)
df_pmt_32_1_1 = convertParquetToDF(pmt_32_1_1)
df_pmt_32_1_2 = convertParquetToDF(pmt_32_1_2)

In [33]:
df_truth_32_1

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,114000400665696,665696,32,1,1,170,170,1.312122e+05,0.045589,2.148992,...,1.844175e+06,1.0,2.700705e-05,8.891057e-35,1286.661743,7780.790527,0.5,456.220154,0,1018.384155
1,115000600138754,138754,32,1,1,93,263,2.677757e+06,3.056381,1.249584,...,1.828938e+05,1.0,1.629861e-04,1.490586e-33,1012.475159,972.081787,0.5,681.626709,0,1059.699829
2,114000400665728,665728,32,1,1,215,478,3.105628e+05,4.643663,2.557549,...,2.281205e+05,1.0,1.880685e-05,5.151188e-34,1215.146362,1062.376099,0.5,672.177246,0,1158.643921
3,115000600138769,138769,32,1,1,394,872,7.291569e+07,1.376329,1.323334,...,1.762801e+05,1.0,6.241383e-04,5.923431e-33,858.934814,1053.857788,0.5,1042.778198,1,793.928467
4,114000400665735,665735,32,1,1,64,936,1.391510e+05,2.550350,1.309242,...,1.812613e+05,1.0,3.725267e-05,3.437636e-34,837.095276,1129.747925,0.5,503.852081,1,291.993256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,115000100009693,9693,32,1,10,104,870320,1.748889e+06,0.177174,2.352001,...,1.459540e+05,1.0,1.973203e-06,3.076522e-34,806.530273,777.197937,0.5,570.022400,0,413.552460
29996,114000501073150,1073150,32,1,10,40,870360,3.005953e+05,4.963473,0.332415,...,2.952687e+05,1.0,8.519232e-05,4.826198e-34,920.709412,2145.045410,0.5,410.646149,1,458.765808
29997,115000100009705,9705,32,1,10,459,870819,2.131569e+06,4.971455,1.715046,...,1.765080e+05,1.0,3.550451e-05,1.232124e-33,976.119202,939.145996,0.5,792.833679,0,989.811951
29998,114000501073211,1073211,32,1,10,73,870892,2.214562e+05,1.991865,2.969661,...,1.931556e+05,1.0,8.019437e-08,3.930128e-35,977.781128,950.651611,0.5,408.476654,0,1009.899963


In [40]:
show_offset_edges(df_truth_32_1)

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
3000,114000400728353,728353,32,1,2,270,270,242373.125000,4.583750,0.655208,...,3.519754e+05,1.0,0.000089,4.248212e-34,1203.685913,2450.924561,0.5,696.306396,1,733.808044
6000,114000400794722,794722,32,1,3,202,202,874777.875000,6.174209,0.066665,...,1.907319e+05,1.0,0.000101,8.842720e-34,934.709656,969.520203,0.5,611.698364,0,520.607666
9000,114000400861207,861207,32,1,4,55,55,852777.187500,3.045562,1.694286,...,1.748917e+05,1.0,0.000023,8.271383e-34,997.619995,900.107483,0.5,469.840393,0,1015.952271
12000,114000100034677,34677,32,1,5,65,65,107504.203125,4.684008,0.733328,...,2.251628e+05,1.0,0.000040,2.973866e-34,1096.557739,1124.577881,0.5,422.857422,0,921.559265
15000,114000100096571,96571,32,1,6,744,744,585471.125000,5.790648,2.780167,...,3.237160e+06,1.0,0.000098,6.093937e-34,954.873169,12524.310547,0.5,881.575134,0,1109.683350
18000,114000100157292,157292,32,1,7,105,105,326993.375000,5.163021,0.586818,...,1.637618e+05,1.0,0.000053,5.416090e-34,966.450500,810.507812,0.5,553.462830,1,445.493500
21000,114000100216921,216921,32,1,8,20,20,196000.109375,4.178150,2.521303,...,1.667288e+05,1.0,0.000001,1.781410e-34,842.768250,763.559265,0.5,228.760635,0,451.044281
24000,114000500946768,946768,32,1,9,27,27,137913.859375,3.288262,2.020029,...,1.923975e+05,1.0,0.000002,2.630268e-34,1106.717651,980.962891,0.5,235.986404,0,1163.440796
27000,114000501010522,1010522,32,1,10,425,425,661501.562500,4.749238,1.704519,...,1.655026e+05,1.0,0.000076,7.716208e-34,855.979370,939.868469,0.5,874.630554,0,926.993713


In [39]:
show_shard_edges(df_truth_32_1)

,event_no,original_event_no,subdirectory_no,part_no,shard_no,N_doms,offset,energy,azimuth,zenith,...,TotalColumnDepthCGS,TotalPrimaryWeight,TotalWeight,TotalXsectionCGS,TrueActiveLengthAfter,TrueActiveLengthBefore,TypeWeight,max_interPMT_distance,isWithinIceCube,lepton_intra_distance
0,114000400665696,665696,32,1,1,170,170,131212.156250,0.045589,2.148992,...,1.844175e+06,1.0,0.000027,8.891057e-35,1286.661743,7780.790527,0.5,456.220154,0,1018.384155
3000,114000400728353,728353,32,1,2,270,270,242373.125000,4.583750,0.655208,...,3.519754e+05,1.0,0.000089,4.248212e-34,1203.685913,2450.924561,0.5,696.306396,1,733.808044
6000,114000400794722,794722,32,1,3,202,202,874777.875000,6.174209,0.066665,...,1.907319e+05,1.0,0.000101,8.842720e-34,934.709656,969.520203,0.5,611.698364,0,520.607666
9000,114000400861207,861207,32,1,4,55,55,852777.187500,3.045562,1.694286,...,1.748917e+05,1.0,0.000023,8.271383e-34,997.619995,900.107483,0.5,469.840393,0,1015.952271
12000,114000100034677,34677,32,1,5,65,65,107504.203125,4.684008,0.733328,...,2.251628e+05,1.0,0.000040,2.973866e-34,1096.557739,1124.577881,0.5,422.857422,0,921.559265
15000,114000100096571,96571,32,1,6,744,744,585471.125000,5.790648,2.780167,...,3.237160e+06,1.0,0.000098,6.093937e-34,954.873169,12524.310547,0.5,881.575134,0,1109.683350
18000,114000100157292,157292,32,1,7,105,105,326993.375000,5.163021,0.586818,...,1.637618e+05,1.0,0.000053,5.416090e-34,966.450500,810.507812,0.5,553.462830,1,445.493500
21000,114000100216921,216921,32,1,8,20,20,196000.109375,4.178150,2.521303,...,1.667288e+05,1.0,0.000001,1.781410e-34,842.768250,763.559265,0.5,228.760635,0,451.044281
24000,114000500946768,946768,32,1,9,27,27,137913.859375,3.288262,2.020029,...,1.923975e+05,1.0,0.000002,2.630268e-34,1106.717651,980.962891,0.5,235.986404,0,1163.440796
27000,114000501010522,1010522,32,1,10,425,425,661501.562500,4.749238,1.704519,...,1.655026e+05,1.0,0.000076,7.716208e-34,855.979370,939.868469,0.5,874.630554,0,926.993713


In [34]:
df_pmt_32_1_1

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,t2,t3,t4,t5,T10,T50,sigmaT,eccentricity_PCA,aspect_contrast_PCA,hypotenuse
0,114000400665696,665696,79.410004,-248.240005,277.630005,-18.810898,-170.445206,88.687332,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,42.474106
1,114000400665696,665696,79.410004,-248.240005,260.609985,-18.810898,-170.445206,71.667313,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,42.474106
2,114000400665696,665696,79.410004,-248.240005,141.470001,-18.810898,-170.445206,-47.472672,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,42.474106
3,114000400665696,665696,210.470001,-209.770004,360.779999,112.249100,-131.975204,171.837326,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,42.474106
4,114000400665696,665696,210.470001,-209.770004,292.700012,112.249100,-131.975204,103.757339,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,42.474106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865040,115000600157471,157471,-88.050003,-384.299988,244.699997,-12.441719,-13.344666,-70.037704,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000
865041,115000600157471,157471,35.540001,-364.829987,412.609985,111.148285,6.125336,97.872284,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000
865042,115000600157471,157471,35.540001,-364.829987,395.589996,111.148285,6.125336,80.852295,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000
865043,115000600157471,157471,35.540001,-364.829987,361.549988,111.148285,6.125336,46.812286,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000


In [35]:
df_pmt_32_1_1['event_no'].nunique()

3000

In [36]:
df_pmt_32_1_2

,event_no,original_event_no,dom_x,dom_y,dom_z,dom_x_rel,dom_y_rel,dom_z_rel,pmt_area,rde,...,t2,t3,t4,t5,T10,T50,sigmaT,eccentricity_PCA,aspect_contrast_PCA,hypotenuse
0,114000400728353,728353,361.000000,-422.829987,261.220001,53.397766,-355.548157,210.288956,0.0444,1.0,...,11223.0,-1.0,-1.0,-1.0,0.0,0.0,19.000000,0.747453,0.201696,242.129913
1,114000400728353,728353,158.940002,-345.230011,57.299999,-148.662231,-277.948181,6.368958,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.747453,0.201696,242.129913
2,114000400728353,728353,158.940002,-345.230011,40.270000,-148.662231,-277.948181,-10.661041,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.747453,0.201696,242.129913
3,114000400728353,728353,282.179993,-325.739990,192.070007,-25.422241,-258.458160,141.138962,0.0444,1.0,...,12234.0,-1.0,-1.0,-1.0,0.0,730.0,365.000000,0.747453,0.201696,242.129913
4,114000400728353,728353,282.179993,-325.739990,158.029999,-25.422241,-258.458160,107.098953,0.0444,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.747453,0.201696,242.129913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851201,115000200039102,39102,22.110001,509.500000,436.579987,29.579372,8.543457,-32.168549,0.0444,1.0,...,10229.0,10257.0,10284.0,10296.0,22.0,516.0,275.486511,-1.000000,-1.000000,-1.000000
851202,115000200039102,39102,22.110001,509.500000,419.559998,29.579372,8.543457,-49.188538,0.0444,1.0,...,10177.0,10191.0,10311.0,10342.0,48.0,381.0,385.208923,-1.000000,-1.000000,-1.000000
851203,115000200039102,39102,22.110001,509.500000,402.540009,29.579372,8.543457,-66.208527,0.0444,1.0,...,10293.0,10877.0,11127.0,11764.0,0.0,602.0,557.180176,-1.000000,-1.000000,-1.000000
851204,115000200039102,39102,22.110001,509.500000,385.519989,29.579372,8.543457,-83.228546,0.0444,1.0,...,11229.0,-1.0,-1.0,-1.0,0.0,0.0,204.000000,-1.000000,-1.000000,-1.000000


In [37]:
df_pmt_32_1_2['event_no'].nunique()

3000